In [ ]:
import numpy as np              # Импортируем библиотеку NumPy для работы с массивами и матрицей
from sklearn.datasets import fetch_openml     # Импортируем модуль для загрузки набора данных
from sklearn.model_selection import train_test_split  # Модуль для разделения данных на тренировочные и тестовые
from sklearn.metrics import accuracy_score      # Модуль для измерения точности модели

class Perceptron:
    """
    Класс, представляющий однослойный перцептрон.
    """
    def __init__(self, learning_rate=0.25, epochs=5):
        """
        Конструктор класса, инициализирующий параметры модели.

        :param learning_rate: Скорость обучения (по умолчанию 0.01)
        :param epochs: Количество эпох обучения (по умолчанию 100)
        """
        self.learning_rate = learning_rate       # Сохраняем значение скорости обучения
        self.epochs = epochs                     # Сохраняем количество эпох обучения
        self.weights = None                      # Начальные веса будут установлены позже
        self.bias = None                         # Смещение будет установлено позже

    def sigmoid(self, x):
        """
        Реализация сигмоидальной функции активации.

        :param x: Входной вектор или скаляр
        :return: Значение сигмоиды от x
        """
        return 1 / (1 + np.exp(-x))             # Формула сигмоидной функции

    def sigmoid_derivative(self, x):
        """
        Реализация производной сигмоидальной функции.

        :param x: Входной вектор или скаляр
        :return: Значение производной сигмоиды от x
        """
        sig = self.sigmoid(x)                    # Сначала находим значение сигмоиды
        return sig * (1 - sig)                   # Производная сигмоиды

    def predict(self, X):
        """
        Функция для предсказания классов на основе текущих весов и смещений.

        :param X: Матрица входных данных (признаки)
        :return: Вектор предсказанных классов (0 или 1)
        """
        linear_output = np.dot(X, self.weights) + self.bias   # Линейный выход = сумма произведений весов и признаков плюс смещение
        probabilities = self.sigmoid(linear_output)           # Применяем сигмоидальную активацию к линейному выходу
        return (probabilities >= 0.5).astype(int)             # Возвращаем классы: 1, если вероятность больше 0.5, иначе 0

    def fit(self, X_train, y_train):
        """
        Обучающая функция модели.

        :param X_train: Тренировочная матрица признаков
        :param y_train: Тренировочные метки классов
        """
        n_samples, n_features = X_train.shape          # Определяем количество образцов и признаков

        # Инициализация весов и смещения нулями
        self.weights = np.zeros(n_features)            # Весов столько же, сколько признаков
        self.bias = 1                                  # Смещение - это один элемент

        for epoch in range(self.epochs):               # Проходимся по заданному количеству эпох
            # Вычисляем линейный выход для каждого образца
            linear_output = np.dot(X_train, self.weights) + self.bias

            # Применяем сигмоидальную активацию к линейному выходу
            predicted = self.sigmoid(linear_output)

            # Вычисляем ошибку между предсказанными значениями и истинными классами
            error = y_train - predicted

            # Вычисляем градиенты для весов и смещения
            dw = (1 / n_samples) * np.dot(X_train.T, error * self.sigmoid_derivative(linear_output))
            db = (1 / n_samples) * np.sum(error * self.sigmoid_derivative(linear_output))

            # Обновляем веса и смещение согласно градиентам и скорости обучения
            self.weights += self.learning_rate * dw
            self.bias += self.learning_rate * db

# Загружаем датасет MNIST
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)   # Загружаем данные MNIST

# Преобразуем метки в бинарные (цифры 0 против всех остальных)
y_binary = (y != '0').astype(int)                             # 0 заменяется на 0, другие цифры на 1

# Делиим данные на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42)              # 20% данных отводится на тестовый набор

# Нормализуем данные (масштабируем пиксельные значения от 0 до 1)
X_train = X_train / 255.0                                     # Делим каждый пиксель на 255 (максимальное значение)
X_test = X_test / 255.0                                       # То же самое делаем для тестового набора

# Создаем и обучаем модель
perceptron = Perceptron()                                      # Создаем экземпляр класса Perceptron
perceptron.fit(X_train, y_train)                              # Запускаем обучение модели

# Прогнозируем результаты на тестовых данных
predictions = perceptron.predict(X_test)                       # Получаем предсказанные классы

# Оцениваем точность модели
accuracy = accuracy_score(y_test, predictions)                 # Сравниваем реальные метки с предсказаниями
print(f'Accuracy: {accuracy:.4f}')                            # Выводим точность модели с точностью до четырех знаков

Accuracy: 0.9041
